# SAMOS Spectroscopy - Visualization with jdaviz

This notebook demonstrates interactive visualization of SAMOS spectra using:
- **Specviz**: For viewing individual 1D spectra
- **Mosviz**: For multi-object spectroscopy with 2D spectra and images

## Prerequisites

You should have already run:
- Notebook 01: Initial inspection (creates `spec_*.fits` files)
- Notebook 04-06: Trace extraction and wavelength calibration (creates `spec1d_*.fits` and `spec2d_*.fits`)

**Note**: This notebook works with the output from the updated pipeline!

In [ ]:
import warnings
warnings.simplefilter('ignore')

import os
from pathlib import Path
from jdaviz import Specviz, Mosviz
from specutils import Spectrum1D

print("✓ Packages loaded")

## Configuration

Set your working directory where the reduced data is located.

In [ ]:
# Target information
target_name = "ABELL3120"
target_mode = "SAMI_manual_Mask_T00_Low_Red"

# Working directory where your reduced data is
analysis_top_directory = "/Users/nestrada/Documents/SAMOS/SAMOS_REDUCED"
working_directory = os.path.join(analysis_top_directory, target_name, target_mode)

# Change to working directory
os.chdir(working_directory)
print(f"Working directory: {working_directory}")

# List available files
spec_files = sorted(Path('.').glob('spec_*.fits'))
spec1d_files = sorted(Path('.').glob('spec1d_*.fits'))
spec2d_files = sorted(Path('.').glob('spec2d_*.fits'))
tile_files = sorted(Path('.').glob('tile_*.fits'))

print(f"\nFound:")
print(f"  {len(spec_files)} spec_*.fits files (from notebook 01)")
print(f"  {len(spec1d_files)} spec1d_*.fits files (1D extracted spectra)")
print(f"  {len(spec2d_files)} spec2d_*.fits files (2D rectified spectra)")
print(f"  {len(tile_files)} tile_*.fits files (cutout images)")

## Specviz - Single Spectrum Viewer

View a single 1D spectrum interactively.

**Features:**
- Pan and zoom
- Measure line properties
- Fit models
- Apply smoothing

See: https://jdaviz.readthedocs.io/en/latest/specviz/

In [ ]:
# Check if 1D spectra exist
if len(spec1d_files) == 0:
    print("⚠️ No spec1d_*.fits files found!")
    print("You need to run notebooks 04-06 to create 1D extracted spectra.")
    print("\nFor now, you can only visualize the raw 2D spectra from notebook 01.")
else:
    # Load first spectrum
    spectrum_file = str(spec1d_files[0])
    print(f"Loading: {spectrum_file}")
    
    # Read with specutils
    spec1d = Spectrum1D.read(spectrum_file)
    
    # Create Specviz instance
    specviz = Specviz()
    specviz.load_data(spec1d, data_label=Path(spectrum_file).stem)
    
    # Display
    specviz.show()

### Load Additional Spectra

Load more spectra for comparison.

In [ ]:
if len(spec1d_files) > 1:
    # Load first 5 spectra (or however many you want)
    n_spectra = min(5, len(spec1d_files))
    
    for i in range(n_spectra):
        spectrum_file = str(spec1d_files[i])
        spec1d = Spectrum1D.read(spectrum_file)
        specviz.load_data(spec1d, data_label=Path(spectrum_file).stem)
        print(f"Loaded: {Path(spectrum_file).stem}")
    
    print(f"\n✓ Loaded {n_spectra} spectra")
    print("Use the data menu in Specviz to select which spectrum to display")
else:
    print("Only one spectrum available")

## Mosviz - Multi-Object Spectroscopy Viewer

View multiple spectra simultaneously with 2D spectra and cutout images.

**Features:**
- Browse through multiple objects
- View 1D spectrum, 2D spectrum, and cutout image side-by-side
- Measure redshifts
- Compare spectra

See: https://jdaviz.readthedocs.io/en/latest/mosviz/

In [ ]:
# Check if we have the necessary files
if len(spec1d_files) == 0 or len(spec2d_files) == 0:
    print("⚠️ Mosviz requires spec1d_*.fits and spec2d_*.fits files")
    print("Run notebooks 04-06 to create these files.")
else:
    # Create Mosviz instance
    mosviz = Mosviz()
    
    # Prepare file lists
    spec1d_list = [str(f) for f in spec1d_files]
    spec2d_list = [str(f) for f in spec2d_files]
    
    # Optionally add cutout images if they exist
    if len(tile_files) > 0:
        tile_list = [str(f) for f in tile_files]
        print(f"Loading {len(spec1d_list)} spectra with cutout images...")
        mosviz.load_data(spectra_1d=spec1d_list, spectra_2d=spec2d_list, images=tile_list)
    else:
        print(f"Loading {len(spec1d_list)} spectra (no cutout images)...")
        mosviz.load_data(spectra_1d=spec1d_list, spectra_2d=spec2d_list)
    
    # Display
    print("\n✓ Data loaded into Mosviz")
    print("Use the navigation controls to browse through objects")
    mosviz.show()

## Tips for Using jdaviz

### Specviz:
- **Pan**: Click and drag
- **Zoom**: Scroll wheel or box zoom button
- **Reset**: Home button
- **Line Analysis**: Use the line analysis plugin
- **Model Fitting**: Use the model fitting plugin
- **Export**: Use the export plugin to save modified spectra

### Mosviz:
- **Navigate**: Use arrow buttons or slider to browse objects
- **Redshift**: Use the redshift slider to match lines
- **Link**: Viewers are linked - changes in one affect others
- **Extract**: Click on 2D spectrum to update 1D extraction region

### Keyboard Shortcuts:
- `h`: Show help
- `r`: Reset view
- Arrow keys: Pan
- `+/-`: Zoom in/out

## Troubleshooting

### Widget not displaying
- Make sure you're in JupyterLab (not classic Jupyter)
- Try: `jupyter labextension list` to check if jdaviz is installed
- Restart kernel and clear outputs

### Files not found
- Check your `working_directory` path
- Make sure you've run the previous notebooks
- Verify files exist: `ls spec*.fits`

### Performance issues
- Try loading fewer spectra at once
- Close other jdaviz instances
- Increase browser memory if needed

## Next Steps

After visualization:
- Export spectra for further analysis
- Measure emission/absorption lines
- Determine redshifts
- Compare with templates
- Create publication-quality plots

For more advanced features, see the jdaviz documentation:
- https://jdaviz.readthedocs.io/